In [0]:
# Import Required Libraries
from pyspark.sql.functions import *

In [0]:
# Read order_item Data from bronze layer
df_order_item = spark.read.format("delta").load("abfss://olist-data@retailds.dfs.core.windows.net/bronze/order_items")

In [0]:
# Display Schema
df_order_item.printSchema()

In [0]:
# Display Sample Data
df_order_item.limit(5).display()

In [0]:
# Count
df_order_item.count()

In [0]:
# Distinct values from order_item_id
df_order_item.select("order_item_id").distinct().count()

In [0]:
# Distinct values from order_id
df_order_item.select("order_id").distinct().count()

In [0]:
# Display Null values from each column
df_order_item.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_order_item.columns
]).display()

In [0]:
# Check for duplicate records
df_order_item.groupBy("order_id","order_item_id").count().filter(col("count")>1).display()

In [0]:
# Typecasting columns 
df_order_item = df_order_item.withColumn("price", col("price").cast("double"))\
    .withColumn("freight_value", col("freight_value").cast("double"))\
        .withColumn("shipping_limit_date", col("shipping_limit_date").cast("timestamp"))

In [0]:
# Display
df_order_item.limit(10).display()

In [0]:
# Adding new item_total_amount column
df_order_item = df_order_item.withColumn(
    "item_total_amount",
    col("price") + col("freight_value")
)

In [0]:
# Display records
df_order_item.limit(10).display()

In [0]:
# Adding new column is_valid_item
df_order_item = df_order_item.withColumn(
    "is_valid_item",
    (col("freight_value") >= 0) & (col("price") >= 0)
)

In [0]:
# Display records
df_order_item.limit(10).display()

In [0]:
# write to delta
df_order_item.write\
    .format('delta')\
        .mode("overwrite")\
            .option("overwriteSchema", "true")\
                .save('abfss://olist-data@retailds.dfs.core.windows.net/silver/order_items')